In [115]:
from gurobipy import *

In [116]:
model = Model()

## Data

In [117]:
# Number of teams
n = 30

# Number of rounds
k = 2 * (n - 1)

# Set of teams
I = [i for i in range(n)]

# Set of rounds
K = [i for i in range(k)]

# Set of best teams (Brazil and Argentina)
Is = [0, 2]

# Set of names
# names = ['BRA', 'ARG', 'PER', 'COL', 'URU', 'BOL', 'VEN', 'ECU', 'PAR', 'CHI']
names = ['ARG', 'BOL', 'BRA', 'CHI', 'COL', 'ECU', 'PAR', 'PER', 'URU', 'VEN']

## Variables

In [118]:
# X_ijk = 1 if team i plays against team j in date k

X = []
for i in range(n):
    team = []
    for j in range(n):
        dates = [None] * k
        for kp in range(k):
            dates[kp] = model.addVar(vtype=GRB.BINARY, name='{}_vs_{}_in_date_{}'.format(names[i % 10] + str(i // 10), names[j % 10] + str(j // 10), kp))
        team.append(dates)
    X.append(team)

## Constraints

### Double robin

In [119]:
# 1. Each team plays against other twice
model.addConstrs(quicksum(X[i][j][kp] + X[j][i][kp] for kp in K if kp < n-1) == 1 for i in I for j in I if i != j);

model.addConstrs(quicksum(X[i][j][kp] + X[j][i][kp] for kp in K if kp >= n-1) == 1 for i in I for j in I if i != j);

# 2. Each team plays once as local against the others
model.addConstrs(quicksum(X[i][j][kp] for kp in K) == 1 for i in I for j in I if i != j);

### Compactness

In [120]:
# 3. Each team plays one match once per date
model.addConstrs(quicksum((X[i][j][kp] + X[j][i][kp]) for i in I if i != j) == 1 for j in I for kp in K);

### Top teams

In [121]:
# 4. Top teams
model.addConstrs(quicksum(X[i][j][k] + X[j][i][k] + X[i][j][k + 1] + X[j][i][k + 1] for j in Is) <= 1
                 for i in I if Is.count(i) == 0 for k in K if k < len(K) - 1);

### Balance

In [122]:
# Let's introduce the auxiliar variable y_ik
# which is equal to 1 when the team i has a H-A sequence in the double rounds
Y = []
for i in range(n):
    y_i = [None] * k
    for kp in range(k):
        y_i[kp] = model.addVar(vtype=GRB.BINARY)
    Y.append(y_i)

# 5. Boundaries for y_ik
model.addConstrs(quicksum(Y[i][k] for k in K if k % 2 == 0) >= n//2 - 1 for i in I);

model.addConstrs(quicksum(Y[i][k] for k in K if k % 2 == 0) <= n//2 for i in I);

# 6.
model.addConstrs(quicksum(X[i][j][k] + X[j][i][k + 1] for j in I if j != i) <= 1 + Y[i][k] for i in I for k in K if k % 2 == 0);

# 7.
model.addConstrs(Y[i][k] <= quicksum(X[i][j][k] for j in I if j != i) for i in I for k in K if k % 2 == 0);

# 8.
model.addConstrs(Y[i][k] <= quicksum(X[j][i][k + 1] for j in I if j != i) for i in I for k in K if k % 2 == 0);

## Objective Function

In [123]:
# Let's introduce the variable w_ik
# which is 1 when the team i has an away break in the double round starting in the round k
W = []
for i in range(n):
    w_i = [None] * k
    for kp in range(k):
        w_i[kp] = model.addVar(vtype=GRB.BINARY)
    W.append(w_i)

# 9.
model.addConstrs(quicksum(X[j][i][k] + X[j][i][k + 1] for j in I if j != i) <= 1 + W[i][k] for i in I for k in K if k % 2 == 0);

# 10. Boundaries for w_ik
model.addConstrs(W[i][k] <= quicksum(X[j][i][k] for j in I if j != i) for i in I for k in K if k % 2 == 0);

model.addConstrs(W[i][k] <= quicksum(X[j][i][k + 1] for j in I if j != i) for i in I for k in K if k % 2 == 0);

# Minimize the w_ik

obj = quicksum(W[i][k] for i in I for k in K if k % 2 == 0);

#model.setParam(GRB.Param.TimeLimit, 60*5)

model.setObjective(obj, GRB.MINIMIZE)
#model.setObjective(quicksum(X[i][j][kp] for i in I for j in I for kp in K), GRB.MINIMIZE)

## Schemes

### Mirrored

In [124]:
#model.addConstrs(X[i][j][kp] == X[j][i][kp + n - 1] for i in I for j in I if i != j for kp in K if kp < n - 1);

### French

In [125]:
#model.addConstrs(X[i][j][0] == X[j][i][-1] for i in I for j in I if i != j);

#model.addConstrs(X[i][j][kp] == X[j][i][kp + n - 2] for i in I for j in I if i != j for kp in K[1:n-1]);

### English

In [126]:
#model.addConstrs(X[i][j][n - 2] == X[j][i][n - 1] for i in I for j in I if i != j);

#model.addConstrs(X[i][j][kp] == X[j][i][kp + n] for i in I for j in I if i != j for kp in K[:n-2]);

### Inverted

In [127]:
model.addConstrs(X[i][j][kp] == X[j][i][2*n - 1 - kp - 2] for i in I for j in I if i != j for kp in K[:n-2]);

### Back to back

In [128]:
#model.addConstrs(X[i][j][kp] == X[j][i][kp + 1] for i in I for j in I if i != j for kp in K[0:len(K):2]);

### Min-Max separation

In [129]:
c = 6
d = 12

#model.addConstrs(quicksum(X[i][j][k2] + X[j][i][k2] for k2 in K if k2 >= kp and k2 <= kp + c) <= 1 for i in I
#                for j in I if i != j for kp in K if kp <= len(K) - c);

#model.addConstrs(quicksum(X[i][j][k2] for k2 in K if k2 >= 0 and (k2 <= kp + d or k2 <= 2 * (n - 1)) and k2 != kp)
#                 >= X[j][i][kp] for i in I for j in I if i != j for kp in K);

### New restrictions [March 6]

In [130]:
# 11. In half of the matches, one as local and one as visitant against the best teams

model.addConstrs(quicksum(X[i][j][kp] for j in Is for kp in K[:n-1]) == 1 for i in I if Is.count(i) == 0);

### New constraints [March 13]

In [131]:
# 12. Balance H-A and A-H sequences

model.addConstrs(quicksum(Y[i][k2] for k2 in range(kp, kp + 3)) <= 2 for i in I for j in I if i != j
                 for kp in K if kp % 2 == 0 and kp < len(K) - 4);

model.addConstrs(quicksum(Y[i][k2] for k2 in range(kp, kp + 3)) >= 1 for i in I for j in I if i != j
                 for kp in K if kp % 2 == 0 and kp < len(K) - 4);

In [132]:
#model.optimize()

In [133]:
model.write('/home/jamerrq/Documents/Dropbox/inverted30.mps')